<a href="https://colab.research.google.com/github/allfed/CropOpt/blob/standardize/notebooks/yield_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# These Instructions Are Only Needed The First Time You Setup CoLab

In [ ]:
!rm -rf allfed-integrated-model/
!git clone -b main https://github.com/allfed/allfed-integrated-model.git

In [ ]:
# This adds the Python files in our repo to our path so we can import
# them

import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path) 

In [ ]:
!pip install PuLP
!pip install numpy
!pip install matplotlib


# Optimizer demo

The purpose of this file is to allow researchers to run the optimizer with differing assumptions. Please reach out to morgan@allfed.info if you have questions about this code.

This will publish all graphs accessible in the recently submitted paper.

First, let's import the Optimizer, which runs through the code to optimize the minimum people fed of calories, fat, and protein. 

In [1]:
import os
import sys
os.chdir('/content/allfed-integrated-model/notebooks/')
module_path = os.path.abspath(os.path.join('..'))
print(module_path)
if module_path not in sys.path:
    sys.path.append(module_path)
import numpy as np

/home/dmrivers/Code/allfed-integrated-model


Now, we run the no resilient food case, generating the total food production with resilient foods.

In [2]:
import src.run_model_no_resilient_foods

/home/dmrivers/.local/lib/python3.9/site-packages/pulp/apis/coin_api.py:89: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")



Estimated people fed, no resilient foods, no waste
7.2441166

Estimated people fed, no resilient foods, minus waste & delayed halt of nonhuman consumption 
5.9240094



Run the no resilient food case

In [3]:
import src.run_model

/home/dmrivers/.local/lib/python3.9/site-packages/pulp/apis/coin_api.py:89: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")



no waste estimated people fed
13.595705

Food available after waste, feed ramp down and biofuel ramp down, with resilient foods
11.3642

Calculating 2100 calorie diet, excess feed to animals
Diet computation complete


Run the baseline scenario

In [3]:
import src.run_model_before_catastrophe

/home/dmrivers/.local/lib/python3.9/site-packages/pulp/apis/coin_api.py:89: UserWarning: Parameter fracGap is being depreciated for gapRel
  warnings.warn("Parameter fracGap is being depreciated for gapRel")






people_fed_billions
16.352426








people_fed_billions
7.744541






Run the Monte Carlo scenarios. We choose the "True" value for run_all_scenarios, which loads existing data from the monte carlo and plots it.

In [4]:
from src.monte_carlo import MonteCarlo

/home/dmrivers/.local/lib/python3.9/site-packages/statsmodels/compat/pandas.py:65: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import Int64Index as NumericIndex


In [5]:
MonteCarlo.run_all_scenarios(10000, 1000, True)

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


95% lower
2372.5260986538465
95% upper
4182.760642307692


/home/dmrivers/.local/lib/python3.9/site-packages/seaborn/_decorators.py:36: FutureWarning: Pass the following variable as a keyword arg: x. From version 0.12, the only valid positional argument will be `data`, and passing other arguments without an explicit keyword will result in an error or misinterpretation.
  warnings.warn(
/home/dmrivers/Code/allfed-integrated-model/src/plotter.py:1460: UserWarning: This figure includes Axes that are not compatible with tight_layout, so results might be incorrect.
  plt.tight_layout()


Import analysis data saved in previous runs, and plot the result for primary production

In [10]:
from src.plotter import Plotter
resilient_food_primary_production_analysis = np.load(
    '../data/resilient_food_primary_analysis.npy',
    allow_pickle=True).item()

no_resilient_food_primary_production_analysis = np.load(
    '../data/no_resilient_food_primary_analysis.npy',
    allow_pickle=True).item()

Plotter.plot_figure_supplement_primary_production_abcd(
    resilient_food_primary_production_analysis,
    no_resilient_food_primary_production_analysis,
    48,
    72)


In [11]:
print(os.getcwd())


/home/dmrivers/Code/allfed-integrated-model/notebooks
